# EC1B1 Macroeconomics Coursework - Code

This notebook contains all the code for our EC1B1 Macroeconomics coursework. The code is divided into the following sections:

## 0. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

## 1. Load and Preprocess Data